In [1]:
import json
import requests

In [2]:
#https://pypi.org/project/gamma-viewer/
from gamma_viewer import GammaViewer
from IPython.display import display

In [3]:
def submit_to_ars(m):
    submit_url = 'https://ars.transltr.io/ars/api/submit'
    response = requests.post(submit_url,json=m)
    try:
        message_id = response.json()['pk']
    except:
        print('fail')
        message_id = None
    print(f'https://arax.ncats.io/?source=ARS&id={message_id}')
    return message_id

In [45]:
def retrieve_ars_results(mid):
    message_url = f'https://ars.transltr.io/ars/api/messages/{mid}?trace=y'
    response = requests.get(message_url)
    j = response.json()
    print( j['status'] )
    results = {}
    for child in j['children']:
        if child['status'] == 'Done':
            childmessage_id = child['message']
            child_url = f'https://ars.transltr.io/ars/api/messages/{childmessage_id}'
            child_response = requests.get(child_url).json()
            try:
                nresults = len(child_response['fields']['data']['message']['results'])
                if nresults > 0:
                    results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}
            except:
                nresults=0
        else:
            nresults = 0
        print( child['status'], child['actor']['agent'],nresults )
    return results

In [5]:
def printjson(j):
    print(json.dumps(j,indent=4))

In [6]:
def name_lookup(text):
    url= f'https://name-resolution-sri.renci.org/lookup?string={text}&offset=0&limit=10'
    response = requests.post(url)
    printjson(response.json())

In [7]:
def one_hop_message(subject_category,object_category,predicate=None,subject_id = None, object_id = None):
    query_graph = {
    "nodes": {
        'a':{
            "category": subject_category
        },
        'b':{
            "category": object_category
        }
    },
    "edges": {
        'ab':{
            "subject": "a",
            "object": "b",
            #"predicate": predicate
        }
    }
    }
    if predicate is not None:
        query_graph['edges']['ab']['predicate'] = predicate
    if subject_id is not None:
        query_graph['nodes']['a']['id'] = subject_id
    if object_id is not None:
        query_graph['nodes']['b']['id'] = object_id
    message = {"message": {"query_graph": query_graph},'knowledge_graph':{"nodes": [], "edges": [],}, 'results':[]}
    return message

In [9]:
name_lookup('glycerol kinase')

{
    "www:bget?hsa:2710": [
        "GKD; glycerol kinase",
        "K00864 glycerol kinase [EC:2.7.1.30] | (RefSeq) GK",
        "GK1"
    ],
    "www:bget?hsa:2712": [
        "GKTA; glycerol kinase 2",
        "K00864 glycerol kinase [EC:2.7.1.30] | (RefSeq) GK2",
        "GKP2"
    ],
    "www:bget?hsa:55750": [
        "MULK; acylglycerol kinase",
        "K09881 acylglycerol kinase [EC:2.7.1.94] | (RefSeq) AGK",
        "CATC5",
        "CTRCT38",
        "MTDPS10"
    ],
    "MONDO:0010613": [
        "glycerol kinase deficiency",
        "glycerol kinase deficiency; GKD",
        "inborn error of glycerol kinase activity",
        "inborn glycerol kinase activity disorder",
        "rare inborn error of glycerol kinase activity",
        "rare inborn error of glycerol kinase activity",
        "GKD",
        "GK deficiency",
        "GK1 deficiency",
        "hyperglycerolemia"
    ],
    "www:bget?ko:K00864": [
        "glycerol kinase [EC:2.7.1.30]",
        "GK",
        "g

In [8]:
m = one_hop_message('biolink:ChemicalSubstance','biolink:Gene',object_id='UniProtKB:P32189')

In [9]:
printjson(m)

{
    "message": {
        "query_graph": {
            "nodes": {
                "a": {
                    "category": "biolink:ChemicalSubstance"
                },
                "b": {
                    "category": "biolink:Gene",
                    "id": "UniProtKB:P32189"
                }
            },
            "edges": {
                "ab": {
                    "subject": "a",
                    "object": "b"
                }
            }
        }
    },
    "knowledge_graph": {
        "nodes": [],
        "edges": []
    },
    "results": []
}


In [10]:
mid = submit_to_ars(m)

https://arax.ncats.io/?source=ARS&id=4cc5b9fa-ff62-4e3b-b0fb-d0b54cf5cd58


In [46]:
j = retrieve_ars_results(mid)

Done
Done ara-aragorn 86
Done ara-arax 50
Done ara-bte 85
Done ara-unsecret 14
Done kp-genetics 0
Error kp-molecular 0
Done ara-explanatory 0
Unknown ara-improving 0
Unknown ara-robokop 0
Done kp-textmining 0
Error ara-ncats 0
Done kp-cam 0
Done ara-aragorn-exp 69


In [19]:
response = requests.post('https://automat.renci.org/kegg/query',json=m)

In [20]:
response.status_code

200

In [21]:
response.json()

{'message': {'query_graph': {'nodes': {'a': {'id': None,
     'category': 'biolink:ChemicalSubstance',
     'is_set': False},
    'b': {'id': 'UniProtKB:P32189',
     'category': 'biolink:Gene',
     'is_set': False}},
   'edges': {'ab': {'subject': 'a',
     'object': 'b',
     'predicate': None,
     'relation': None}}},
  'knowledge_graph': {'nodes': {}, 'edges': {}},
  'results': []},
 'knowledge_graph': {'nodes': [], 'edges': []},
 'results': []}

In [22]:
m = one_hop_message('biolink:ChemicalSubstance','biolink:Gene',object_id='NCBIGene:2710')

In [23]:
response = requests.post('https://automat.renci.org/kegg/query',json=m)

In [24]:
response.json()

{'message': {'query_graph': {'nodes': {'a': {'id': None,
     'category': 'biolink:ChemicalSubstance',
     'is_set': False},
    'b': {'id': 'NCBIGene:2710', 'category': 'biolink:Gene', 'is_set': False}},
   'edges': {'ab': {'subject': 'a',
     'object': 'b',
     'predicate': None,
     'relation': None}}},
  'knowledge_graph': {'nodes': {'CHEBI:15422': {'category': ['biolink:NamedThing',
      'biolink:MolecularEntity',
      'biolink:Entity',
      'biolink:ChemicalSubstance',
      'biolink:BiologicalEntity'],
     'name': 'ATP',
     'attributes': [{'type': 'NA',
       'value': ["adenosine 5'-(tetrahydrogen triphosphate)",
        'ATP',
        "Adenosine 5'-triphosphate",
        'Adenosine triphosphate',
        "ADENOSINE-5'-TRIPHOSPHATE",
        'H4atp'],
       'name': 'synonyms',
       'url': None,
       'source': None},
      {'type': 'NA',
       'value': 'ATP',
       'name': 'name',
       'url': None,
       'source': None},
      {'type': 'NA',
       'value': [

In [25]:
mid = submit_to_ars(m)

https://arax.ncats.io/?source=ARS&id=5dbcac7b-05c2-4e46-9a50-3bb4c75cac2e


In [26]:
j = retrieve_ars_results(mid)

Running
Done ara-aragorn 86
Done ara-arax 50
Done ara-bte 85
Done ara-unsecret 14
Done kp-genetics 0
Error kp-molecular 0
Done ara-explanatory 0
Unknown ara-improving 0
Unknown ara-robokop 0
Done kp-textmining 0
Error ara-ncats 0
Done kp-cam 0
Done ara-aragorn-exp 69


In [30]:
j = retrieve_ars_results('9aa08c33-0cd0-4401-b834-f2b59b585f14')

Done
{
  "message": "9aa08c33-0cd0-4401-b834-f2b59b585f14",
  "status": "Done",
  "actor": {
    "pk": 1,
    "channel": "general",
    "agent": "ara-aragorn",
    "path": "runquery"
  },
  "children": []
}


In [34]:
child_url = f'https://ars.transltr.io/ars/api/messages/3e32f299-c335-4b79-9b3e-443661bc626f'
child_response = requests.get(child_url).json()

In [35]:
printjson(child_response)

{
    "model": "tr_ars.message",
    "pk": "3e32f299-c335-4b79-9b3e-443661bc626f",
    "fields": {
        "name": "",
        "code": 200,
        "status": "Done",
        "actor": 1,
        "timestamp": "2021-03-12T14:50:01.358Z",
        "data": {
            "message": {
                "query_graph": {
                    "nodes": {
                        "n1": {
                            "id": null,
                            "category": [
                                "biolink:Pathway"
                            ],
                            "is_set": false
                        },
                        "n0": {
                            "id": [
                                "NCBIGENE:1017"
                            ],
                            "category": [
                                "biolink:Gene"
                            ],
                            "is_set": false
                        }
                    },
                    "edges": {
 

In [41]:
r = requests.get("https://api.bte.ncats.io/v1/predicates")
rj = r.json()

In [43]:
rj['biolink:Gene']['biolink:Pathway']

['biolink:participates_in', 'biolink:has_participants']

In [47]:
m = one_hop_message('biolink:ChemicalSubstance','biolink:Disease',object_id='MONDO:0010613')

In [ ]:
mid_ = submit_to_ars(m)